In [1]:
# setup colab
!git clone https://github.com/Mark-Kac-Center/cna2023-critical-brain.git
import sys
sys.path.insert(0,'cna2023-critical-brain')
!ln -s cna2023-critical-brain/example_data example_data

Cloning into 'cna2023-critical-brain'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 309 (delta 45), reused 55 (delta 33), pack-reused 237
Receiving objects: 100% (309/309), 38.43 MiB | 5.34 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [32]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # for printing progress bars

# The concept of criticality

## 1.1 Magnetization
Throughout the workshop we will use a module `workshops` containing useful function. Use the `workshops.IsingModel` class to simulate an Ising model for several values of temperature on a 20x25 lattice. You should create a two-dimensional lattice for the model using the `workshops.Grid.grid_2d` method.

Take the temperatures in an interval `[0.1, 4.0]`, decide how many points to take based on the simulation time for a single temp. You have to specify the number of time steps (1000 should be enough), and create a simple loop to aggregate the results.

**Tip**: IsingModel class has a simple method for simulation: `simulate()`

The magnetization at time *t* can be calculated as an average over all spins:
$$M = \frac{1}{N}\sum_i \sigma_i $$
Plot the final time-averaged magnetization vs the temperature. You should get something like:
![image](plots/ising_magnetization.png)

**Tip**: In case you are running out of time to run the simulation (it takes ~5 min on a laptop/colab for 20 temp values), you can load simulation data `example_data/ising_dynamics.npz` with `np.load`.

In [ ]:
#### code here

## 1.2 Snapshots
To see a detailed picture of the Ising system on a microscopic level, plot two or three states of the lattice ("snapshots") for small, large and moderate temperatures. Where is the transition? Can you identify it by looking at the snapshots?

The results should look similarly to these:
![image](plots/ising_snap1.png)
![image](plots/ising_snap2.png)
![image](plots/ising_snap3.png)


In [2]:
#### code here

## 1.3 Binomial model (*)
Consider the Binomial Model, a variation of the Ising model where the spins are independent of each other and each is drawn from a specially crafted binomial distribution. 

Investigate the magnetization in this case. Use the same temperature interval as before but decrease the step to `0.05` and set a smaller `n_steps=100`. Can you spot significant differences between the Ising and Binomial models? What happened here, is our Binomial model critical?

To ensure you have a complete picture, plot the snapshots and discuss differences between the Binomial and Ising models.

**Tip**: In case you are running out of time to run the simulation (it takes ~5 min on a laptop/colab), you can load simulation data for the Binomial model `example_data/binomial_dynamics.npz` with `np.load`.

In [4]:
# a Binomial model
from workshops import IsingModel

class BinomialModel(IsingModel):
    
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.n_transient = 0
        
    def sweep(self,s: dict) -> None:
        p = 1/2*(self.calc_mag(T = self.T,J = self.J)+1)
        
        for n in s.keys():
            s[n] = -(2*np.random.binomial(n=1,p=p)-1)
        

In [ ]:
#### code here

# Towards a brain model



## 2.1 Haimovici model

We turn to the Haimovici model of the brain confusingly called `workshops.SERModel`. To make it run, we to load a graph on which the neurons live. We use a healthy human connectome (or adjacency matrix) `hagmann_connectome.npz`.  Plot it to see how the connections are distributed (using `plt.spy`). What do you see clearly?

Run the Haimovici model simulation for `n_steps=2000` time steps (or more if you wish!). The threshold parameter should vary between `[0.01,0.2]`.

The output matrix nodes can have 3 states:
* active (excited) nodes are represented by 1,
* refractory nodes are represented by -1,
* inactive (susceptible) nodes are represented by 0.

Plot a set of magnetizations for each sub-population of neurons using the function `plt.fill_between` similar to this: 

![image](plots/brain_magnetization.png)

Provide correct labels for each neuron population with `plt.text`. Can you spot the transition? Is it critical?


In [20]:
#### code here

## 2.2 Temperature or threshold? 

The Ising model has a temperature while the Haimovici model has a threshold parameter. Both are confusingly called T but are they similar in interpretation?

Perhaps the easiest is to plot temporal dynamics for both models in all three sub-, critical- and super-critical parameter regimes. To this end, pick a single spin/neuron. What do you observe as the temperature changes?

**Tip**: Plots of the spin sub-populations in the Ising model (similar to 2.1) can also be helpful.

In [ ]:
#### code here

## 2.3 Artificial connectomes

OK, so what about artificial connectomes? Instead of human connectome, you can try Watts-Strogatz: write a function generating an artificial connectome based on the weighted Watts-Strogatz graph. Draw the distribution of weights from an exponential distribution with scale factor of $1/12.5$ (this approximates the weights found in a real human connectome).

The Watts-Strogatz graphs can be easily created using `nx.watts_strogatz_graph` and `nx.adjacency_matrix` to extract the adjacency matrix.

Create two *Watts-Strogatz connectomes* with 2000 nodes; one with mean number of neighbours `k=10`, second with `k=2`. The *rewiring* probability should be around `p=0.5`.

Try plotting the fractions of active/refractory/inactive nodes creating a plot similar to one above. Are there any differences?

**Tip**: If you run into problems with creating your own routine for the Watts-Strogatz connectome, you can use the function `workshops.watts_strogatz_connectome`.

In [21]:
#### code here

# Criticality in the Haimovici model



## 3.1 Clusters in the Ising model
Find the clusters in the Ising model. Use the previously used grid graph in Ex. 1.1 and the Ising model snapshots plotted in Ex. 1.2. 

**Tips**:
In case you cannot figure out how to start on your own, we suggest a path to follow:
1) The adjacency matrix `nx.adjacency_matrix(network).toarray()` and a flatten snapshot `snapshot.flatten()` have matching indices. 
2) Use the flattened snapshot to form a binary mask of the adjacency matrix which leaves only the connections where both spins are alike.
3) From the resulting masked adjacency matrix, form a new graph using `nx.from_numpy_array(adj)`.
4) Find connected components of the graph via `nx.connected_components`.
5) The result is a list of index sets being part of the clusters. Warning: they're not ordered!

Take the Ising model snapshot near $T = T_c$. Using the algorithm above, plot using `plt.imshow` the snapshot and color the largest cluster. How large is the second-largest cluster in comparison? What about $ T < T_c$? 

**Tip**: If this part was too hard, we provide a function `workshops.plot_color_clusters` which does the cluster coloring for you.

In [27]:
#### code here

## 3.2 Criticality indicators in the Haimovici model

Find the cluster sizes in the Haimovici model. Use a predefined function `workshops.batch_clusters`. Try plotting the average size of the largest cluster vs. threshold. What is the difference between the largest cluster and the second largest cluster?

Find the standard deviation of the total activity. Next prepare a function which computes the autocorrelation coefficient at lat $\tau=1$. Do these quantities have something in common?

**Tip**: Autocorrelation can be found using `np.corrcoef` and `np.roll`.

In [26]:
#### code here

## 3.3 Detective work
A numpy file `example_data/mystery_simulation.npz` contains simulation data from an unknown source. The file also contains indicators of criticality like the cluster sizes, autocorrelation and standard deviation of the activity. By inspecting these quantities and based on previous examples, can you infer the most likely criticality status of this uknown system? What is the deciding factor?

In [164]:
#### code here

# Playing around



## 4.1 Lobotomy
Try to disconnect the hemispheres in the human connectome. Note that left hemisphere is represented by nodes from 0 to 498, right hemisphere by nodes 499 to 998.

`Spy` on the adjacency matrix to ensure your procedure was correct. Then, using the connectome with disconnected hemispheres run Haimovici simulations and check the criticality using clusters and other methods (e.g. std. dev. of activity).

In [ ]:
#### code here

## 4.2 Stroke
Artificial Stroke: remove connections between Sensory-Motor RSN (labelled by 4) and the rest of the brain. Check the criticality of the system using activity measure, e.g. standard deviation of the total activity, and compare with result computed using second largest cluster size.


Try also other labels:

VisM (Medial Visual) - 1
VisL (Lateral Visual) - 2
Aud (Auditory) - 3
SM (Sensory-Motor) - 4
DMN (Default Mode Network) - 5
EC (Executive Control) - 6
DorL (Dorsal Visual Stream Left) - 7
DorR (Dorsal Visual Stream Right) - 8




In [ ]:
#### code here

## 4.3 Epilepsy
Enchance connectome connections by a constant value. Find the criticality using different measures and compare them with the healthy connectome.

In [29]:
#### code here